In [3]:
!pip install wordcloud
!pip install nltk
!pip install missingno

# for data
import pandas as pd

# for missing values
import missingno as msno

# for plotting
import matplotlib.pyplot as plt

# for word frequency
from wordcloud import WordCloud

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

     |████████████████████████████████| 366 kB 13.3 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
df = pd.read_csv("spend_data.csv")

# Exploratory Data Analysis (EDA)

## General observations
We have mostly categorical data in our dataset apart from the 'value' attribute, which is re-scaled by min-max normalization. (The 'Unnamed' column is not a feature, it contains the row indices). The text field attribute contains unstructured text data. Missing attribute values: in case of columns 1, 5 and 6, more than 99% of the data is missing -> I dropped these attributes. I dropped further attributes based on domain knowledge as they are not (always) available before classification.
The remaining missing values in 'division' attribute are proven to be MAR (Missing at Random -> likelihood of missing is related to some of the observed data -> in our case profit_center). MAR is safe to remove the missing data without producing bias in the model and we have a large enough dataset to do that so we dropped the related rows.

In [ ]:
# check the dimensions of the data

print(df.shape)
df.head()

In [ ]:
df.columns

In [7]:
# check data from the value coloumn -> normalized by min-max

df['value'].describe()

count    411688.000000
mean          0.000180
std           0.003071
min           0.000000
25%           0.000004
50%           0.000009
75%           0.000033
max           1.000000
Name: value, dtype: float64

In [ ]:
# calculate missing values statistics

def null_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns
    
miss_values = null_values(df)
miss_values.head(10)

In [9]:
# drop columns based on missing values statistics -  > 99% of missing values

df = df.drop(columns=['col1', 'col5', 'col6'])

print(df.shape)

(411688, 19)


In [ ]:
# drop columns based on domain knowledge -> , 
# subcategory, GL account data are not (always) known prior the classification

df = df.drop(columns=['col7','col8','col9','col10','col11','col12','col13','col14','col15'])

print(df.shape)
df.head()

In [ ]:
# check the missing values in 'division' -> MAR, MCAR or MNAR?

msno.matrix(df)

In [ ]:
# MAR -> The likelihood of being missing depends on profit_center data -> it is safe to remove the missing data and we have a large enough
# dataset to do that

msno.matrix(df.sort_values('col2'))

In [ ]:
# drop missing values

df = df.dropna()
df.shape

## Sample questions

### Class distribution
We clearly have an imbalanced class problem here where the number of observations per class are not equally distributed. E.g., the number of purchase item requests belonging to "office supplies" is more than 8 650 times of "mass storage" items. As conventional algorithms are often biased towards the majority class(es) this can lead to low accuracy in minority classes. In our case the value distribution adds further complexity as not the most populated classes account for the highest value. (The domain-experts expressed the wish for the higher value purchase requests to be classified more accuratly.)
There are several approaches to handle the imbalance class problem like cost function based approaches and sampling based approaches.

In [ ]:
# 1. sample question: How many items are in which class? -> imbalanced classes

# item count by classes
print(df['group'].value_counts())

# plot it as a histogram
print(df['group'].value_counts().sort_values().plot(kind='barh',colormap='Paired',
                                                    y='value', x='group', 
                                                    legend=False, title= "Item count by classes"))


In [ ]:
# 3. sample question: How is the value distribution per class?
# item count distribution per class -><- aggregated value distribution per class

# aggregated value per classes
print(df.groupby(['group'], sort=False).agg({'value': sum}).sort_values('value',ascending=False))


# plot it as a histogram
df_flat = df.groupby(['group'])['value'].sum().reset_index()
print(df_flat.sort_values(['value'], ascending=True).plot(kind='barh',colormap='Pastel2',
                                                     y='value', x='group', 
                                                     legend=False, title= "Aggregated value per class"))
 


## Class distribution (After text attribute "transformation")
After doing the necessary preprocessing we can further explore our text data.
We can observe the most salient word tokens for each class with using wordclouds as visualization.
Each class has its own key-token set, which we can leverage in our classification model.
However we can see that there are words which are prominent in more classes e.g. "service" in 'Scanner' and 'Server'
classes seems frequent -> this can lead to misclassification if we don't handle it (e.g. we can filter out words/terms which
are salient in more classes).

In [ ]:
# load the pre-processed data

df = pd.read_csv("out/cleaned_data.csv")

print(df.shape)
df.head()

In [ ]:
# create wordclouds per classes

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        max_words=50,
        max_font_size=50, 
        scale=3,
        collocations=False,
        random_state=1
    ).generate(str(data).replace("'",' '))

    fig = plt.figure(1, figsize=(8,8))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()



df.groupby('group').apply(lambda x: show_wordcloud(x.english_wostopw.tolist(), 
                                                                         title=f"{x.name}"))

In [ ]:
# create word frequency table

df_most_common3 = df.groupby('group')['english_wostopw'].apply(lambda x: 
                                                               nltk.FreqDist(nltk.tokenize.word_tokenize(' '.join(
                                                                                        x))).most_common(3))

print(df_most_common3)